---
## 0. Imports

In [1]:
from bs4 import BeautifulSoup
import http.client
import urllib
import os

import pandas as pd
import numpy as np
import csv

import dateutil.parser
import datetime
import sys

import spotipy
import spotipy.util

In [2]:
filepath = ('./01_successful-charts-pulls'
            + '/pull_D-2019-11-19_T-19-56-59-692877'
            + '/Result_D-2019-11-19_T-19-56-59-693112_FrmD-2017-01-01_ToD-2019-11-18.csv')

chartCrawl_df = pd.read_csv(filepath, index_col=0)
chartCrawl_df['Date'] = pd.to_datetime(chartCrawl_df['Date'], yearfirst=True)

---
## 1. Spotify API

### 1.1. Connection

In [3]:
client_id = '987f8aab8f804962a2f19a86e310905c'
client_secret = 'bdb457608ae84339ad7d3c41696cf10e'
cred_manager = spotipy.util.oauth2.SpotifyClientCredentials(client_id, client_secret)

In [4]:
sp = spotipy.Spotify(client_credentials_manager=cred_manager)

### 1.2. Contents

In [5]:
def gen_type_str(obj, r_pat=' '):
    type_str = str(type(obj)).rpartition(r_pat)[2][:-1].replace('\'', '').strip()
    if isinstance(obj, str) and ('http' in obj):
        return '(url str)'
    else:
        return '('+type_str+')'

def print_dict_keys(print_arg, lvl=0, spc_mult=5, r_mrgn=50):
    if isinstance(print_arg, list) and lvl==0:
        print_arg = print_arg[0]
    for key, item in print_arg.items():
        indt_lvl = (spc_mult * lvl)
        key_spacing = ' '*indt_lvl
        type_spacing = ' '*(r_mrgn-len(key)-indt_lvl)
        print(key_spacing, key, type_spacing, gen_type_str(item))
        if isinstance(item, dict):
            print_dict_keys(item, lvl+1)
        else:
            continue

In [6]:
sf_track_id = '3JWiDGQX2eTlFvKj3Yssj3'
sf_album_id = sp.track(sf_track_id)['album']['id']
fmt_str = '{:_^70}'
sp_api_dict = {
    'TRACK DETAILS':[sp.track, sf_track_id],
    'AUDIO FEATURES':[sp.audio_features, sf_track_id],
    'AUDIO ANALYSIS':[sp.audio_analysis, sf_track_id],
    'ALBUM DETAILS':[sp.album, sf_album_id],
}
for sp_name, sp_list in sp_api_dict.items():
    print('\n', fmt_str.format(sp_name), '\n')
    print_dict_keys(sp_list[0](sp_list[1]))


 ____________________________TRACK DETAILS_____________________________ 

 album                                               (dict)
      album_type                                     (str)
      artists                                        (list)
      available_markets                              (list)
      external_urls                                  (dict)
           spotify                                   (url str)
      href                                           (url str)
      id                                             (str)
      images                                         (list)
      name                                           (str)
      release_date                                   (str)
      release_date_precision                         (str)
      total_tracks                                   (int)
      type                                           (str)
      uri                                            (str)
 artists                   

---
## 2. Helper Methods

In [7]:
def gen_internalTrackId_df(chartCrawl_df_arg):
    """Returns a DataFrame containing all unique internal track ids."""
    interalId_srs = chartCrawl_df_arg['Spotify_URL'].str.rpartition('/')[2].unique()
    return pd.DataFrame(interalId_srs, columns=['Internal Track ID'], dtype=str)

In [8]:
internalTrackId_df = gen_internalTrackId_df(chartCrawl_df)
internalTrackId_df

,Internal Track ID
0,4Km5HrUvYTaSUfiSGPJeQR
1,343YBumqHu19cGoGARUTsd
2,5aAx2yezTd8zXrkmtKl66Z
3,7BKLCZ1jbUBVqRi2FVlTVw
4,6fujklziTHa8uoM5OQSfIo
5,7yyRTcZmCiyzzJlNzGC9Ol
6,1xznGGDReH1oQq0xzbwXa3
7,7FB8l7UA1HKqnuSLjP9qDc
8,4pdPtRcBmOSQDlJ3Fk945m
9,0SGkqnVQo9KPytSri1H6cF


---
## 3. API Methods

<b>
<font color="red">  

## TO-DO:  
1. Audio Analysis  
2. Audio Features 
    
</font>
</b>

In [9]:
def get_trackDetails(internalTrackId_row):
    internalTrackId = internalTrackId_row['Internal Track ID']
    trackDetails_dict = sp.track(internalTrackId)
    return {'Internal Track ID':internalTrackId, 'Track Details Obj':trackDetails_dict}

In [10]:
def get_audioFeatures(internalTrackId_row):
    internalTrackId = internalTrackId_row['Internal Track ID']
    audioFeatures_list = sp.audio_features(internalTrackId)
    audioFeatures_dict = audioFeatures_list[0]
    return {'Internal Track ID':internalTrackId, 'Audio Features Obj':audioFeatures_dict}

In [11]:
def get_audioAnalysis(internalTrackId_row):
    internalTrackId = internalTrackId_row['Internal Track ID']
    audioAnalysis_list = sp.audio_analysis(internalTrackId)
    audioAnalysis_dict = audioAnalysis_list['track']
    return {'Internal Track ID':internalTrackId, 'Audio Analysis Obj':audioAnalysis_dict}

In [12]:
def gen_trackInfo_df(chartCrawl_df,
                     get_track_details=False,
                     get_audio_features=False,
                     get_audio_analysis=False):
    """Returns and saves to csv the pd.DataFrames of track details."""
    internalTrackId_df = gen_internalTrackId_df(chartCrawl_df)
    
    trackDetails_df = internalTrackId_df.apply(get_trackDetails, 
                                               axis=1, 
                                               result_type='expand') if get_track_details else None
    audioFeatures_df = internalTrackId_df.apply(get_audioFeatures, 
                                                axis=1, 
                                                result_type='expand') if get_audio_features else None
    audioAnalysis_df = internalTrackId_df.apply(get_audioAnalysis, 
                                                axis=1, 
                                                result_type='expand') if get_audio_analysis else None
    
    return (trackDetails_df, audioFeatures_df, audioAnalysis_df)

In [13]:
trackInfo_tup = gen_trackInfo_df(chartCrawl_df[chartCrawl_df['Artist']=='Drake'],
                                 get_track_details=True,
                                 get_audio_features=True,
                                 get_audio_analysis=False)

In [14]:
drake_details_df = trackInfo_tup[0]

In [15]:
tgt_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

In [47]:
drake_features_df = pd.DataFrame(trackInfo_tup[1], copy=True)
for feature in tgt_features:
    drake_features_df[feature] = drake_features_df['Audio Features Obj'].apply(lambda a: a[feature])
drake_features_df.set_index('Internal Track ID')
drake_features_df.to_csv('./drake_audio_features.csv')

In [17]:
talk_up_df = drake_details_df[(drake_details_df['Internal Track ID']=='3Yw09dj3cTXsAzlLjgBfIP') |
                 (drake_details_df['Internal Track ID']=='4ksuI04WMvUnJbHQjgs3L5')]
talk_up_df

,Internal Track ID,Track Details Obj
53,3Yw09dj3cTXsAzlLjgBfIP,"{'album': {'album_type': 'album', 'artists': [..."
86,4ksuI04WMvUnJbHQjgs3L5,"{'album': {'album_type': 'album', 'artists': [..."


In [18]:
gods_plan_df = drake_details_df[drake_details_df['Internal Track ID'].isin(
    ['2XW4DbS6NddZxRPm5rMCeY', '6T8cJz5lAqGer9GUHGyelE', '2VWbHHhWnMzKWPUs4IEEW9', '6DCZcSspjsKoFjzjrWoCdn'] 
)]
gods_plan_df

,Internal Track ID,Track Details Obj
40,2XW4DbS6NddZxRPm5rMCeY,"{'album': {'album_type': 'single', 'artists': ..."
45,6T8cJz5lAqGer9GUHGyelE,"{'album': {'album_type': 'single', 'artists': ..."
56,2VWbHHhWnMzKWPUs4IEEW9,"{'album': {'album_type': 'album', 'artists': [..."
76,6DCZcSspjsKoFjzjrWoCdn,"{'album': {'album_type': 'album', 'artists': [..."


In [19]:
dont_talk_df = drake_details_df[drake_details_df['Internal Track ID'].isin(
    ['36ONiya0OANYknz0GgJmwB', '6G8kHiVZ1jW7vHMPVRNZU0'] 
)]
dont_talk_df

,Internal Track ID,Track Details Obj
55,36ONiya0OANYknz0GgJmwB,"{'album': {'album_type': 'album', 'artists': [..."
75,6G8kHiVZ1jW7vHMPVRNZU0,"{'album': {'album_type': 'album', 'artists': [..."


In [20]:
tgt_keys_arr = ['disc_number', 'duration_ms', 'explicit', 'external_ids', 
                'external_urls', 'href', 'id', 'name', 'preview_url', 'track_number', 'uri']

def print_track_compares(track_df, keys_arr=tgt_keys_arr):
    print('----------\n')
    for track_idx, details_obj in track_df['Track Details Obj'].items():
        print('track_idx:\t', track_idx, '\n')
        for tgt_key in tgt_keys_arr:
            print(tgt_key + ':\t\t', details_obj[tgt_key])
        print('\n----------\n')

In [21]:
print_track_compares(talk_up_df)

----------

track_idx:	 53 

disc_number:		 1
duration_ms:		 195813
explicit:		 True
external_ids:		 {'isrc': 'USCM51800194'}
external_urls:		 {'spotify': 'https://open.spotify.com/track/3Yw09dj3cTXsAzlLjgBfIP'}
href:		 https://api.spotify.com/v1/tracks/3Yw09dj3cTXsAzlLjgBfIP
id:		 3Yw09dj3cTXsAzlLjgBfIP
name:		 Talk Up
preview_url:		 None
track_number:		 11
uri:		 spotify:track:3Yw09dj3cTXsAzlLjgBfIP

----------

track_idx:	 86 

disc_number:		 1
duration_ms:		 223240
explicit:		 True
external_ids:		 {'isrc': 'USCM51800232'}
external_urls:		 {'spotify': 'https://open.spotify.com/track/4ksuI04WMvUnJbHQjgs3L5'}
href:		 https://api.spotify.com/v1/tracks/4ksuI04WMvUnJbHQjgs3L5
id:		 4ksuI04WMvUnJbHQjgs3L5
name:		 Talk Up (feat. Jay-Z)
preview_url:		 None
track_number:		 11
uri:		 spotify:track:4ksuI04WMvUnJbHQjgs3L5

----------



In [22]:
print_track_compares(gods_plan_df)

----------

track_idx:	 40 

disc_number:		 1
duration_ms:		 198960
explicit:		 True
external_ids:		 {'isrc': 'USCM51800004'}
external_urls:		 {'spotify': 'https://open.spotify.com/track/2XW4DbS6NddZxRPm5rMCeY'}
href:		 https://api.spotify.com/v1/tracks/2XW4DbS6NddZxRPm5rMCeY
id:		 2XW4DbS6NddZxRPm5rMCeY
name:		 God's Plan
preview_url:		 None
track_number:		 1
uri:		 spotify:track:2XW4DbS6NddZxRPm5rMCeY

----------

track_idx:	 45 

disc_number:		 1
duration_ms:		 198960
explicit:		 True
external_ids:		 {'isrc': 'USCM51800004'}
external_urls:		 {'spotify': 'https://open.spotify.com/track/6T8cJz5lAqGer9GUHGyelE'}
href:		 https://api.spotify.com/v1/tracks/6T8cJz5lAqGer9GUHGyelE
id:		 6T8cJz5lAqGer9GUHGyelE
name:		 God's Plan
preview_url:		 None
track_number:		 1
uri:		 spotify:track:6T8cJz5lAqGer9GUHGyelE

----------

track_idx:	 56 

disc_number:		 1
duration_ms:		 198973
explicit:		 True
external_ids:		 {'isrc': 'USCM51800004'}
external_urls:		 {'spotify': 'https://open.spotify.com/tra

In [23]:
print_track_compares(dont_talk_df)

----------

track_idx:	 55 

disc_number:		 2
duration_ms:		 245786
explicit:		 False
external_ids:		 {'isrc': 'USCM51800208'}
external_urls:		 {'spotify': 'https://open.spotify.com/track/36ONiya0OANYknz0GgJmwB'}
href:		 https://api.spotify.com/v1/tracks/36ONiya0OANYknz0GgJmwB
id:		 36ONiya0OANYknz0GgJmwB
name:		 Don’t Matter To Me
preview_url:		 None
track_number:		 10
uri:		 spotify:track:36ONiya0OANYknz0GgJmwB

----------

track_idx:	 75 

disc_number:		 2
duration_ms:		 245786
explicit:		 False
external_ids:		 {'isrc': 'USCM51800208'}
external_urls:		 {'spotify': 'https://open.spotify.com/track/6G8kHiVZ1jW7vHMPVRNZU0'}
href:		 https://api.spotify.com/v1/tracks/6G8kHiVZ1jW7vHMPVRNZU0
id:		 6G8kHiVZ1jW7vHMPVRNZU0
name:		 Don’t Matter To Me (with Michael Jackson)
preview_url:		 None
track_number:		 10
uri:		 spotify:track:6G8kHiVZ1jW7vHMPVRNZU0

----------



In [24]:
trackInfo_tup[1]

,Audio Features Obj,Internal Track ID
0,"{'danceability': 0.928, 'energy': 0.481, 'key'...",343YBumqHu19cGoGARUTsd
1,"{'danceability': 0.791, 'energy': 0.619, 'key'...",1xznGGDReH1oQq0xzbwXa3
2,"{'danceability': 0.806, 'energy': 0.558, 'key'...",4ckuS4Nj4FZ7i3Def3Br8W
3,"{'danceability': 0.804, 'energy': 0.648, 'key'...",11KJSRSgaDxqydKYiD2Jew
4,"{'danceability': 0.607, 'energy': 0.476, 'key'...",6F609ICg9Spjrw1epsAnpa
5,"{'danceability': 0.852, 'energy': 0.553, 'key'...",27GmP9AWRs744SzKcpJsTZ
6,"{'danceability': 0.724, 'energy': 0.456, 'key'...",1WRy5l3TwrwlzNmB5bOeKw
7,"{'danceability': 0.903, 'energy': 0.62, 'key':...",2FBUoWkIuQXwayw2RNo5l6
8,"{'danceability': 0.833, 'energy': 0.431, 'key'...",433P7tDcIAi6NLnf4Sh6tI
9,"{'danceability': 0.745, 'energy': 0.473, 'key'...",3NxAG2ni1lLa8RKL6a0INc


In [25]:
trackInfo_tup[2]

---
## ... Testing

In [26]:
sp.audio_analysis('2QpGZOhTCHHiKmpSO9FW4h')['track'].keys()

dict_keys(['num_samples', 'duration', 'sample_md5', 'offset_seconds', 'window_seconds', 'analysis_sample_rate', 'analysis_channels', 'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'codestring', 'code_version', 'echoprintstring', 'echoprint_version', 'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version'])

In [27]:
sp.audio_features('2QpGZOhTCHHiKmpSO9FW4h')[0].keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [28]:
sp.audio_analysis('https://open.spotify.com/track/2QpGZOhTCHHiKmpSO9FW4h')

{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1575048778,
  'analysis_time': 3.29412,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 2306237,
  'duration': 104.59125,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 0.0,
  'start_of_fade_out': 104.59125,
  'loudness': -3.503,
  'tempo': 180.291,
  'tempo_confidence': 0.351,
  'time_signature': 4,
  'time_signature_confidence': 1.0,
  'key': 1,
  'key_confidence': 0.03,
  'mode': 1,
  'mode_confidence': 0.378,
  'codestring': 'eJxVmQmy3TgOBK-iI3Bf7n-xySw-tz3dEY5HiSJBoFAo8LfRzhh3fOVro9SxV73fGuW7u7Y6z53fve2bd62zy_1q34t3o57Z9_jq6nm56yqHFRi1W2btp82vrbm-ekfbpa3BCz6ds-5R1u1f74s927yHl_cbpZRvtjnnbqt9o0_-KaPVcS9v127fZmZnj_0NberYU-uq-5u18W0ps7MHv_rAsjYb24z5zcXmYy4eHPZdpWLG2Ke01uvHuQ4nYOPGyXdvh09v6W3X8eENT4uHOPH6Tmwcp-Oas7_T2fbsU2-fu3_MOjxbd1Y-

In [29]:
sp.track('https://open.spotify.com/track/2QpGZOhTCHHiKmpSO9FW4h')

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'name': 'Kanye West',
    'type': 'artist',
    'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
  'available_markets': ['AD',
   'AE',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PS',
   'PT',
   'PY',
   'QA',
   'RO',
   'SA',
   'SE',
   'SG',
   'SK',
 

In [30]:
search_result = sp.search(q='Dance Monkey Tones and I', limit=1, type='track', market='US')
search_result['tracks']['items'][0].keys()

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [31]:
sp.search(q='Dance Monkey Tones and I', limit=3, type='track', market='US')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Dance+Monkey+Tones+and+I&type=track&market=US&offset=0&limit=3',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2NjfBq1NflQcKSeiDooVjY'},
       'href': 'https://api.spotify.com/v1/artists/2NjfBq1NflQcKSeiDooVjY',
       'id': '2NjfBq1NflQcKSeiDooVjY',
       'name': 'Tones and I',
       'type': 'artist',
       'uri': 'spotify:artist:2NjfBq1NflQcKSeiDooVjY'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/0UywfDKYlyiu1b38DRrzYD'},
     'href': 'https://api.spotify.com/v1/albums/0UywfDKYlyiu1b38DRrzYD',
     'id': '0UywfDKYlyiu1b38DRrzYD',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/b98ddadfe65507738699fa7f80dc654f7f4d022d',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/2610e3524ce08c4d6ad16e7b9327f46998e1b821',
       'width': 300},
      {'height': 64,
       'u

In [32]:
drake_url_test1 = sp.track('0w1ZtnzQmtmuuoKxHT0pLL')
drake_url_test1

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
    'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
    'id': '3TVXtAsR1Inumwj472S9r4',
    'name': 'Drake',
    'type': 'artist',
    'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/2o9McLtDM7mbODV7yZF2mc'},
  'href': 'https://api.spotify.com/v1/albums/2o9McLtDM7mbODV7yZF2mc',
  'id': '2o9McLtDM7mbODV7yZF2mc',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273f9a9abe8ed9ffaba55412384',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02f9a9abe8ed9ffaba55412384',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851f9a9abe8ed9ffaba55412384',
    'width': 64}],
  'name': 'Scorpion',
  'release_date': '2018-06-29',
  'release_date_precisi

In [33]:
drake_url_test2 = sp.track('4HG1YiGBseVKzjyKcmAJen')
drake_url_test2

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
    'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
    'id': '3TVXtAsR1Inumwj472S9r4',
    'name': 'Drake',
    'type': 'artist',
    'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}],
  'available_markets': ['AD',
   'AE',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PS',
   'PT',
   'PY',
   'QA',
   'RO',
   'SA',
   'SE',
   'SG',
   'SK',
   'SV

In [34]:
keys_list = list(drake_url_test1.keys())
for key in keys_list:
    print(key)
    print(drake_url_test1[key], '\n')

album
{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'}, 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4', 'id': '3TVXtAsR1Inumwj472S9r4', 'name': 'Drake', 'type': 'artist', 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}], 'available_markets': [], 'external_urls': {'spotify': 'https://open.spotify.com/album/2o9McLtDM7mbODV7yZF2mc'}, 'href': 'https://api.spotify.com/v1/albums/2o9McLtDM7mbODV7yZF2mc', 'id': '2o9McLtDM7mbODV7yZF2mc', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273f9a9abe8ed9ffaba55412384', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02f9a9abe8ed9ffaba55412384', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d00004851f9a9abe8ed9ffaba55412384', 'width': 64}], 'name': 'Scorpion', 'release_date': '2018-06-29', 'release_date_precision': 'day', 'total_tracks': 25, 'type': 'album', 'uri': 'spotify:album:2

In [35]:
pd.set_option('display.max_colwidth', -1)
pd.reset_option('display.max_rows')
drake_url_test1_list = [drake_url_test1[key] for key in drake_url_test1.keys()]
drake_url_test2_list = [drake_url_test2[key] for key in drake_url_test2.keys()]
test_df = pd.DataFrame({'0w1ZtnzQmtmuuoKxHT0pLL':drake_url_test1_list, 
                        '4HG1YiGBseVKzjyKcmAJen':drake_url_test2_list},
                       index=drake_url_test1.keys())
test_df['Same val?'] = test_df['0w1ZtnzQmtmuuoKxHT0pLL']==test_df['4HG1YiGBseVKzjyKcmAJen']

In [36]:
test_df

,0w1ZtnzQmtmuuoKxHT0pLL,4HG1YiGBseVKzjyKcmAJen,Same val?
album,"{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'}, 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4', 'id': '3TVXtAsR1Inumwj472S9r4', 'name': 'Drake', 'type': 'artist', 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}], 'available_markets': [], 'external_urls': {'spotify': 'https://open.spotify.com/album/2o9McLtDM7mbODV7yZF2mc'}, 'href': 'https://api.spotify.com/v1/albums/2o9McLtDM7mbODV7yZF2mc', 'id': '2o9McLtDM7mbODV7yZF2mc', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273f9a9abe8ed9ffaba55412384', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02f9a9abe8ed9ffaba55412384', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d00004851f9a9abe8ed9ffaba55412384', 'width': 64}], 'name': 'Scorpion', 'release_date': '2018-06-29', 'release_date_precision': 'day', 'total_tracks': 25, 'type': 'album', 'uri': 'spotify:album:2o9McLtDM7mbODV7yZF2mc'}","{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'}, 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4', 'id': '3TVXtAsR1Inumwj472S9r4', 'name': 'Drake', 'type': 'artist', 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/1ATL5GLyefJaxhQzSPVrLX'}, 'href': 'https://api.spotify.com/v1/albums/1ATL5GLyefJaxhQzSPVrLX', 'id': '1ATL5GLyefJaxhQzSPVrLX', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/e6d17a95cac4810507dfb0a8242e4463d4c32a30', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/9f5fa5dfc5e084427eb4627a87bfafb2f200e3a4', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ed0e8b6ee4bcdb0565e99bdeb3465eabc8b25bb3', 'width': 64}], 'name': 'Scorpion', 'release_date': '2018-06-29', 'release_date_precision': 'day', 'total_tracks': 25, 'type': 'album', 'uri': 'spotify:album:1ATL5GLyefJaxhQzSPVrLX'}",False
artists,"[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'}, 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4', 'id': '3TVXtAsR1Inumwj472S9r4', 'name': 'Drake', 'type': 'artist', 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}]","[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'}, 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4', 'id': '3TVXtAsR1Inumwj472S9r4', 'name': 'Drake', 'type': 'artist', 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}]",True
available_markets,[],"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, CH, CL, CO, CR, CY, CZ, DE, DK, DO, DZ, EC, EE, EG, ES, FI, FR, GB, GR, GT, HK, HN, HU, ID, IE, IL, IN, IS, IT, JO, JP, KW, LB, LI, LT, LU, LV, MA, MC, MT, MX, MY, NI, NL, NO, NZ, OM, PA, PE, PH, PL, PS, PT, PY, QA, RO, SA, SE, SG, SK, SV, TH, TN, TR, TW, US, UY, VN, ZA]",False
disc_number,2,2,True
duration_ms,247893,247893,True
explicit,False,False,True
external_ids,{'isrc': 'USCM51800224'},{'isrc': 'USCM51800224'},True
external_urls,{'spotify': 'https://open.spotify.com/track/0w1ZtnzQmtmuuoKxHT0pLL'},{'spotify': 'https://open.spotify.com/track/4HG1YiGBseVKzjyKcmAJen'},False
href,https://api.spotify.com/v1/tracks/0w1ZtnzQmtmuuoKxHT0pLL,https://api.spotify.com/v1/tracks/4HG1YiGBseVKzjyKcmAJen,False
id,0w1ZtnzQmtmuuoKxHT0pLL,4HG1YiGBseVKzjyKcmAJen,False


In [37]:
drake_url_test1['id'] == drake_url_test2['id']

False

In [38]:
drake_url_test1['href'] == drake_url_test2['href']

False

In [39]:
drake_url_test1['href']

'https://api.spotify.com/v1/tracks/0w1ZtnzQmtmuuoKxHT0pLL'

In [40]:
drake_url_test1['external_urls'] == drake_url_test2['external_urls']

False

In [41]:
drake_url_test1['external_urls']

{'spotify': 'https://open.spotify.com/track/0w1ZtnzQmtmuuoKxHT0pLL'}

In [42]:
drake_url_test2['external_urls']

{'spotify': 'https://open.spotify.com/track/4HG1YiGBseVKzjyKcmAJen'}

In [43]:
drake_url_test1['external_urls'] == drake_url_test2['external_urls']

False